In [1]:
import pandas as pd

data = pd.read_csv("df_all.csv").sample(1000)
text = list(data.tweet)

/home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt

In [3]:
from ipywidgets import IntProgress


## Sentiment Sample

### Load Sample

In [4]:
data_sample = pd.read_csv('data_full_sentiment.csv')


/home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Clean Sample and put Weight on Tweets

In [5]:
def cleaning_sample(csv_path):
    dict_to_numeric = {"negative": -1, "neutral": 0, "positive": 1}
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=['Unnamed: 0',
                                                'Unnamed: 0.1',
                                                "id", 
                                                "conversation_id", 
                                                "user_id", 
                                                "language", 
                                                "hashtags", 
                                                "avg_len_of_tweet",
                                                "replies_count"
                                      ])
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"],   errors='coerce')
    df_party_name["date"] = pd.to_datetime(df_party_name["date"],  errors='coerce')
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name["sentiment"].replace(dict_to_numeric, inplace=True)
    df_party_name = df_party_name.reset_index(drop=True)
    return df_party_name

In [6]:
df_sample = cleaning_sample('data_full_sentiment.csv')
df_sample

/tmp/ipykernel_14698/3155152343.py:1: DtypeWarning: Columns (1,9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  df_sample = cleaning_sample('data_full_sentiment.csv')


,party,created_at,date,tweet,retweets_count,likes_count,sentiment
0,AFD,2021-08-23 01:59:21,2021-08-23,@sunny_sommer Mit Verlaub: was haben Sie gerau...,0.0,3.0,-1
1,AFD,2021-08-23 01:58:37,2021-08-23,"Ist zwar schlimm, aber insofern egal, weil ja ...",0.0,0.0,-1
2,AFD,2021-08-23 01:54:06,2021-08-23,@UweJunge Wieso sind sie als Abgeordneter eige...,0.0,0.0,-1
3,AFD,2021-08-23 01:54:00,2021-08-23,@ServusTV_News 2015 wiederholt sich - statt Me...,0.0,0.0,0
4,AFD,2021-08-23 01:53:00,2021-08-23,@EggMarkus @ryhminsimon @zrthstr1 @thinks_abou...,0.0,0.0,0
...,...,...,...,...,...,...,...
394749,SPD,2021-08-15 19:04:42,2021-08-15,"@OlafScholz Sie ist so dramatisch, weil Ihr so...",0.0,26.0,0
394750,SPD,2021-08-15 19:04:36,2021-08-15,@stran_d @BILD @spdde @spdbt @Die_Gruenen Das ...,0.0,0.0,0
394751,SPD,2021-08-15 19:04:00,2021-08-15,@Ver_Nunft @NativeYodel @theoneandbodi @FontiF...,0.0,0.0,1
394752,SPD,2021-08-15 19:03:40,2021-08-15,@PhilRobinWeber @spdde @Die_Gruenen Das perfid...,0.0,1.0,0


### Function: Engineer Weighted Sentiment Feature

In [7]:
def sample_feature_engineering(df_sample):
    '''
    Generates the following features: "Weighted Sentiment", "Share of positive tweets", "Share of negative tweets".
    '''
    # Generate "Weighted Sentiment"
    df_sample["likes_count"] = df_sample["likes_count"]+10
    df_sample["retweets_count"] = df_sample["retweets_count"]+10
    df_sample["weighted_sentiment"] = np.log10(df_sample["likes_count"]) * np.log10(df_sample["retweets_count"]) * df_sample["sentiment"]

    # Generate "Share of positive tweets"
    df_sample["share_of_positive_tweets"] = df_sample["sentiment"]    
    dict_only_positive = {-1: 0, 0: 0, 1: 1}
    df_sample["share_of_positive_tweets"].replace(dict_only_positive, inplace=True)
    
    # Generate "Share of negative tweets"
    df_sample["share_of_negative_tweets"] = df_sample["sentiment"]
    dict_only_negative = {-1: 1, 0: 0, 1: 0}
    df_sample["share_of_negative_tweets"].replace(dict_only_negative, inplace=True)
    
    # Return DF
    return df_sample

In [8]:
df_sample_features = sample_feature_engineering(df_sample)


In [43]:
def create_grouped_features(df_sample_features):
    df_sample_features["share_of_positive_tweets2"] = df_sample_features["share_of_positive_tweets"]
    df_sample_features["share_of_negative_tweets2"] = df_sample_features["share_of_negative_tweets"]
    df_sample_features = df_sample_features.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
        "weighted_sentiment": "mean",
        "share_of_positive_tweets": "sum",
        "share_of_positive_tweets2": "count",
        "share_of_negative_tweets": "sum",
        "share_of_negative_tweets2": "count",})
    df_sample_features["share_of_positive_tweets"] = df_sample_features["share_of_positive_tweets"] / df_sample_features["share_of_positive_tweets2"]
    df_sample_features["share_of_negative_tweets"] = df_sample_features["share_of_negative_tweets"] / df_sample_features["share_of_negative_tweets2"]
    df_sample_features = df_sample_features.drop(columns=["share_of_positive_tweets2", "share_of_negative_tweets2"])
    return df_sample_features

In [44]:
df_grouped_sample_features = create_grouped_features(df_sample_features)

In [45]:
df_grouped_sample_features

weighted_sentiment  share_of_positive_tweets  \
created_at party                                                  
2021-08-15 AFD              -0.551851                  0.054348   
           CDU              -0.552591                  0.041535   
           FDP              -0.399836                  0.056978   
           GRUENE           -0.384697                  0.066622   
           LINKE            -0.378079                  0.053360   
           SPD              -0.487310                  0.001898   
2021-08-16 AFD              -0.637995                  0.046294   
           CDU              -0.460671                  0.044845   
           FDP              -0.437177                  0.045623   
           GRUENE           -0.390661                  0.052051   
           LINKE            -0.397315                  0.050900   
           SPD              -0.495646                  0.039770   
2021-08-17 AFD              -0.638291                  0.041417   
           CDU              -0.442927                  0.040910   
           FDP              -0.429510                  0.052316   
           GRUENE           -0.366312                  0.042090   
           LINKE            -0.391006                  0.062731   
           SPD              -0.440405                  0.036463   
2021-08-18 AFD              -0.627671                  0.051325   
           CDU              -0.440914                  0.047931   
           FDP              -0.411528                  0.050710   
           GRUENE           -0.370665                  0.055270   
           LINKE            -0.416017                  0.046655   
           SPD              -0.448519                  0.046303   
2021-08-19 AFD              -0.581999                  0.052531   
           CDU              -0.442530                  0.049242   
           FDP              -0.422964                  0.056050   
           GRUENE           -0.367273                  0.063260   
           LINKE            -0.370792                  0.061602   
           SPD              -0.412294                  0.052698   
2021-08-20 AFD              -0.647927                  0.062451   
           CDU              -0.464322                  0.055595   
           FDP              -0.516874                  0.055597   
           GRUENE           -0.349912                  0.062418   
           LINKE            -0.363787                  0.071565   
           SPD              -0.442151                  0.060363   
2021-08-21 AFD              -0.630686                  0.062205   
           CDU              -0.465397                  0.056433   
           FDP              -0.471214                  0.063120   
           GRUENE           -0.396239                  0.056463   
           LINKE            -0.416131                  0.046154   
           SPD              -0.452435                  0.054945   
2021-08-22 AFD              -0.648703                  0.056489   
           CDU              -0.444479                  0.058855   
           FDP              -0.446072                  0.051710   
           GRUENE           -0.363130                  0.065023   
           LINKE            -0.359615                  0.068129   
           SPD              -0.417378                  0.058851   
2021-08-23 AFD              -0.488925                  0.068273   
           CDU              -0.429284                  0.044593   
           FDP              -0.414451                  0.057223   
           GRUENE           -0.363172                  0.061178   
           LINKE            -0.386060                  0.052454   
           SPD              -0.429584                  0.045611   
2021-08-24 AFD              -0.406045                  0.170213   
           FDP              -0.425377                  0.017544   
           GRUENE           -0.279951                  0.095238   
           LINKE            -0.296296                  0.074074   

## Poll Sample

In [11]:
data_poll = pd.read_csv('pools_data_2021_v2.csv')
data_poll

,Date,Surveyed_Persons,SPD,AfD,Linke,FDP,Grüne,CDU/CSU,other
0,2021-04-26,10039.00,15.0,9.00,6.0,11.00,29.00,24.00,6.0
1,2021-04-27,2082.00,16.0,12.00,8.0,12.00,23.00,23.00,6.0
2,2021-04-28,1689.67,13.0,9.67,8.0,8.67,26.33,7.33,27.0
3,2021-04-29,1042.00,10.0,24.00,13.0,6.00,12.00,0.00,35.0
4,2021-04-30,1643.00,14.0,11.00,8.0,11.00,25.00,24.00,7.0
...,...,...,...,...,...,...,...,...,...
116,2021-08-20,1179.00,11.0,21.00,10.0,6.00,7.00,0.00,45.0
117,2021-08-21,1265.50,16.5,16.50,8.5,9.50,12.00,11.00,26.0
118,2021-08-22,1352.00,22.0,12.00,7.0,13.00,17.00,22.00,7.0
119,2021-08-23,2119.00,23.0,11.00,7.0,13.00,17.00,23.00,6.0


## Pandas Engineer Features Table

### Cleaning Function

In [12]:
def cleaning(party, csv_path):
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=[#'Unnamed: 0',
                              "timezone",
                              "username", 
                              "name", 
                              "place", 
                              "urls", 
                              "photos", 
                              "cashtags", 
                              "retweet", 
                              "quote_url",
                              "video",
                              "thumbnail", 
                              "near", 
                              "source", 
                              "user_rt_id", 
                              "user_rt", 
                              "retweet_id", 
                              "reply_to",  
                              "retweet_date", 
                              "translate", 
                              "trans_src", 
                              "trans_dest",
                              "link",
                              "geo",
                              "time",
                              "mentions"
                                 ])
    df_party_name = df_party_name[(df_party_name.created_at.str.len() == 23) | (df_party_name.created_at.str.len() == 24)]
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"])
    df_party_name["date"] = pd.to_datetime(df_party_name["date"])
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name = df_party_name[(df_party_name.language == "de")]
    df_party_name = df_party_name.drop_duplicates()
    df_party_name.insert(loc=0, column="party", value=party)
    df_party_name = df_party_name.reset_index(drop=True)

    return df_party_name

In [13]:
df_cdu = cleaning("CDU",'../raw_data/drive-download-20210826T100733Z-001/CDU_2308_v1.csv')

/tmp/ipykernel_14698/3831213768.py:1: DtypeWarning: Columns (10,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_cdu = cleaning("CDU",'../raw_data/drive-download-20210826T100733Z-001/CDU_2308_v1.csv')


In [14]:
df_afd = cleaning("AFD",'../raw_data/drive-download-20210826T100733Z-001/afd_real.csv')

/tmp/ipykernel_14698/1516433831.py:1: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df_afd = cleaning("AFD",'../raw_data/drive-download-20210826T100733Z-001/afd_real.csv')


In [15]:
df_fdp = cleaning("FDP", '../raw_data/drive-download-20210826T100733Z-001/fdp.csv')

In [16]:
df_gruene = cleaning("GRUENE", '../raw_data/drive-download-20210826T100733Z-001/gruene.csv')

/tmp/ipykernel_14698/581910460.py:1: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df_gruene = cleaning("GRUENE", '../raw_data/drive-download-20210826T100733Z-001/gruene.csv')


In [17]:
df_linke = cleaning("LINKE", '../raw_data/drive-download-20210826T100733Z-001/linke.csv')

In [18]:
df_spd = cleaning("SPD", '../raw_data/drive-download-20210826T100733Z-001/spd.csv')

### Concat Function

In [19]:
liste = [df_afd, df_cdu, df_fdp, df_gruene, df_linke, df_spd]

In [20]:
def concat(liste):
    df_all = pd.concat(liste)
    df_all = df_all.reset_index(drop=True)
    df_all["avg_len_of_tweet"] = df_all["tweet"].str.len()
    return df_all

In [21]:
df_all = concat(liste)

### Feature Engineering

In [22]:
df_all

,party,id,conversation_id,created_at,date,user_id,tweet,language,replies_count,retweets_count,likes_count,hashtags,Unnamed: 0,avg_len_of_tweet
0,AFD,1429944076061839367,1429944076061839367,2021-08-24 01:10:09,2021-08-24,1425768268866936835,Das war mein Kanzler! Und jetzt wähle ich AfD....,de,0.0,0.0,0.0,['afd'],NaN,160
1,AFD,1429943745571696642,1429538422125961221,2021-08-24 01:08:50,2021-08-24,1130242490492182528,@Phnixauschemni1 @Rosenro88130026 @AfD @Alice_...,de,0.0,0.0,0.0,[],NaN,283
2,AFD,1429943687635689473,1427927572977958919,2021-08-24 01:08:36,2021-08-24,241166049,@MarkPrecht @AfD Du wählst die AfD also nicht?,de,0.0,0.0,0.0,[],NaN,46
3,AFD,1429943518034862080,1427927572977958919,2021-08-24 01:07:56,2021-08-24,241166049,"@codeecke @AfD Du weißt anscheinend nicht, wie...",de,0.0,0.0,0.0,[],NaN,89
4,AFD,1429943276895936520,1429943276895936520,2021-08-24 01:06:58,2021-08-24,241166049,"Sehr schönes Video! Wenn #Grüne mitregieren, ...",de,0.0,0.0,0.0,"['grüne', 'lockdown', 'afd', 'deutschlandabern...",NaN,274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322688,SPD,1426953005061091331,1426949316401410056,2021-08-15 19:04:42,2021-08-15,46327787,"@OlafScholz Sie ist so dramatisch, weil Ihr so...",de,1.0,0.0,26.0,[],NaN,81
322689,SPD,1426952979190530049,1426948683170467845,2021-08-15 19:04:36,2021-08-15,1325444836611289090,@stran_d @BILD @spdde @spdbt @Die_Gruenen Das ...,de,1.0,0.0,0.0,[],NaN,196
322690,SPD,1426952826740162562,1425852582073876484,2021-08-15 19:04:00,2021-08-15,1176581940838903808,@Ver_Nunft @NativeYodel @theoneandbodi @FontiF...,de,1.0,0.0,0.0,[],NaN,359
322691,SPD,1426952744515080201,1426518324482134018,2021-08-15 19:03:40,2021-08-15,312038412,@PhilRobinWeber @spdde @Die_Gruenen Das perfid...,de,0.0,0.0,1.0,[],NaN,186


In [23]:
def pandas_feature_creation(df):
    
    # Create: "Avg. len per tweet of each party"
    df_temp = df.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean"
    }) 
    
    # Create: share of tweets that a party has in comparison to all tweets on a given day 
    df_temp_2 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "tweet": "count"}).groupby(level=0).apply(lambda x: x/x.sum())
    
    # Create: Share of tweets that come from a unique user for each party on a given day
    df_temp_3 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "user_id": "nunique",
    "tweet": "count"})
    df_temp_3["share_unique_users"] = df_temp_3["user_id"] / df_temp_3["tweet"]
    df_temp_3 = df_temp_3["share_unique_users"]
    
     # Join the different temporary DFs into a final DataFrame
    df_final = df_temp.join(df_temp_2).join(df_temp_3)
    df_final = df_final.rename(columns={'tweet': "share_of_tweets"})
    
    return df_final
    

### Table with pandas features

In [39]:
data_features = pandas_feature_creation(df_all)


KeyError: 'The grouper name created_at is not found'

In [40]:
data_features.shape

(59, 6)

In [25]:
df_all = data_features.join(df_grouped_sample_features)

## RNN Model Table

In [26]:
def create_rnn_final_table(data_poll,df_all):
    data_poll = data_poll.rename(columns = {"Date": "created_at", "Linke": "LINKE", "AfD" : "AFD", "Grüne" : "GRUENE", "CDU/CSU":"CDU"})
    data_poll["created_at"] = pd.to_datetime(data_poll["created_at"])
    data_poll = data_poll.set_index("created_at")
    
    data_poll = pd.DataFrame(data_poll.T.unstack(level = 0))
    data_poll.index = data_poll.index.set_names(['created_at', 'party'])
    
    df_final = data_poll.join(df_all, how = "outer")
    
    df_final = df_final.rename(columns = {0: "Polls"})
    
    return df_final
    

In [27]:
df_final = create_rnn_final_table(data_poll,df_all)

In [28]:
# data_poll = data_poll.rename(columns = {"Date": "created_at", "Linke": "LINKE", "AfD" : "AFD", "Grüne" : "GRUENE", "CDU/CSU":"CDU"})
# data_poll["created_at"] = pd.to_datetime(data_poll["created_at"])
# data_poll = data_poll.set_index("created_at")

In [29]:
# df_pool = pd.DataFrame(data_poll.T.unstack(level = 0))

In [30]:
# df_pool.index = df_pool.index.set_names(['created_at', 'party'])

In [31]:
# df_final = df_pool.join(df_all, how = "outer")

In [32]:
# df_final = df_final.rename(columns = {0: "Polls"})

In [53]:
df_final.unstack

<bound method DataFrame.unstack of                               Polls  replies_count  retweets_count  \
created_at party                                                     
2021-04-26 AFD                  9.0            NaN             NaN   
           CDU                 24.0            NaN             NaN   
           FDP                 11.0            NaN             NaN   
           GRUENE              29.0            NaN             NaN   
           LINKE                6.0            NaN             NaN   
...                             ...            ...             ...   
2021-08-24 GRUENE              18.0           15.0             5.0   
           LINKE                6.0            5.0             1.0   
           SPD                 23.0           31.0            18.0   
           Surveyed_Persons  2504.0            NaN             NaN   
           other                9.0            NaN             NaN   

                             likes_count  avg_len_of_t

In [34]:
df_final.to_csv("df_mergedv2.csv")